In [6]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [7]:
data1 = [{'Name':'Jhon','ID':21.528,'Add':'USA'},{'Name':'Joe','ID':3.69,'Add':'USA'},{'Name':'Tina','ID':2.48,'Add':'IND'},{'Name':'Jhon','ID':22.22, 'Add':'USA'},{'Name':'Joe','ID':5.33,'Add':'INA'}]
a = sc.parallelize(data1)
b = spark.createDataFrame(a)
#b.show()

In [9]:
data2 = [{'Name':'Jhon','ID':21.528,'Add':'USA'},{'Name':'Joe','ID':3.69,'Add':'USeA'},{'Name':'Tina','ID':2.48,'Add':'IND'},{'Name':'Jhon','ID':22.22, 'Add':'USdA'},{'Name':'Joe','ID':5.33,'Add':'rsa'}]
c = sc.parallelize(data2)
d = spark.createDataFrame(c)

In [13]:
f = d.join(b,d.Add == b.Add)
f.explain(True)

== Parsed Logical Plan ==
Join Inner, (Add#6 = Add#0)
:- LogicalRDD [Add#6, ID#7, Name#8], false
+- LogicalRDD [Add#0, ID#1, Name#2], false

== Analyzed Logical Plan ==
Add: string, ID: double, Name: string, Add: string, ID: double, Name: string
Join Inner, (Add#6 = Add#0)
:- LogicalRDD [Add#6, ID#7, Name#8], false
+- LogicalRDD [Add#0, ID#1, Name#2], false

== Optimized Logical Plan ==
Join Inner, (Add#6 = Add#0)
:- Filter isnotnull(Add#6)
:  +- LogicalRDD [Add#6, ID#7, Name#8], false
+- Filter isnotnull(Add#0)
   +- LogicalRDD [Add#0, ID#1, Name#2], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [Add#6], [Add#0], Inner
   :- Sort [Add#6 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(Add#6, 200), ENSURE_REQUIREMENTS, [id=#136]
   :     +- Filter isnotnull(Add#6)
   :        +- Scan ExistingRDD[Add#6,ID#7,Name#8]
   +- Sort [Add#0 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(Add#0, 200), ENSURE_REQUIREMENTS, [id=#137]
   

In [ ]:
f.show()

In [16]:
from pyspark.sql.functions import broadcast
f = d.join(broadcast(b),d.Add == b.Add)
f.explain(True)

== Parsed Logical Plan ==
Join Inner, (Add#6 = Add#0)
:- LogicalRDD [Add#6, ID#7, Name#8], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [Add#0, ID#1, Name#2], false

== Analyzed Logical Plan ==
Add: string, ID: double, Name: string, Add: string, ID: double, Name: string
Join Inner, (Add#6 = Add#0)
:- LogicalRDD [Add#6, ID#7, Name#8], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [Add#0, ID#1, Name#2], false

== Optimized Logical Plan ==
Join Inner, (Add#6 = Add#0), rightHint=(strategy=broadcast)
:- Filter isnotnull(Add#6)
:  +- LogicalRDD [Add#6, ID#7, Name#8], false
+- Filter isnotnull(Add#0)
   +- LogicalRDD [Add#0, ID#1, Name#2], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [Add#6], [Add#0], Inner, BuildRight, false
   :- Filter isnotnull(Add#6)
   :  +- Scan ExistingRDD[Add#6,ID#7,Name#8]
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [id=#161]
      +- Filter isnotnull(Ad

In [17]:
f.show()

+---+------+----+---+------+----+
|Add|    ID|Name|Add|    ID|Name|
+---+------+----+---+------+----+
|USA|21.528|Jhon|USA| 22.22|Jhon|
|USA|21.528|Jhon|USA|  3.69| Joe|
|USA|21.528|Jhon|USA|21.528|Jhon|
|IND|  2.48|Tina|IND|  2.48|Tina|
+---+------+----+---+------+----+



In [21]:
peopleDF = (
  ("andrea", "medellin"),
  ("rodolfo", "medellin"),
  ("abdul", "bangalore")
)

peopleDF_a = sc.parallelize(peopleDF)
peopleDF_b = spark.createDataFrame(peopleDF_a, ["first_name", "city"])

peopleDF_b.show()

+----------+---------+
|first_name|     city|
+----------+---------+
|    andrea| medellin|
|   rodolfo| medellin|
|     abdul|bangalore|
+----------+---------+



In [22]:
citiesDF = (
  ("medellin", "colombia", 2.5),
  ("bangalore", "india", 12.3)
)




citiesDF_a = sc.parallelize(citiesDF)
citiesDF_b = spark.createDataFrame(citiesDF_a, ("city", "country", "population"))

citiesDF_b.show()

+---------+--------+----------+
|     city| country|population|
+---------+--------+----------+
| medellin|colombia|       2.5|
|bangalore|   india|      12.3|
+---------+--------+----------+



In [31]:
peopleDF_b_j = peopleDF_b.join(
  broadcast(citiesDF_b),
  peopleDF_b.city == citiesDF_b.city
)

peopleDF_b_j.explain(True)

== Parsed Logical Plan ==
Join Inner, (city#112 = city#124)
:- LogicalRDD [first_name#111, city#112], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [city#124, country#125, population#126], false

== Analyzed Logical Plan ==
first_name: string, city: string, city: string, country: string, population: double
Join Inner, (city#112 = city#124)
:- LogicalRDD [first_name#111, city#112], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [city#124, country#125, population#126], false

== Optimized Logical Plan ==
Join Inner, (city#112 = city#124), rightHint=(strategy=broadcast)
:- Filter isnotnull(city#112)
:  +- LogicalRDD [first_name#111, city#112], false
+- Filter isnotnull(city#124)
   +- LogicalRDD [city#124, country#125, population#126], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [city#112], [city#124], Inner, BuildRight, false
   :- Filter isnotnull(city#112)
   :  +- Scan ExistingRDD[first_name#111,city#112]
   +- Broadca

In [34]:
peopleDF_b_j = peopleDF_b.join(
  citiesDF_b,
  peopleDF_b.city == citiesDF_b.city
)

peopleDF_b_j.explain(True)

== Parsed Logical Plan ==
Join Inner, (city#112 = city#124)
:- LogicalRDD [first_name#111, city#112], false
+- LogicalRDD [city#124, country#125, population#126], false

== Analyzed Logical Plan ==
first_name: string, city: string, city: string, country: string, population: double
Join Inner, (city#112 = city#124)
:- LogicalRDD [first_name#111, city#112], false
+- LogicalRDD [city#124, country#125, population#126], false

== Optimized Logical Plan ==
Join Inner, (city#112 = city#124)
:- Filter isnotnull(city#112)
:  +- LogicalRDD [first_name#111, city#112], false
+- Filter isnotnull(city#124)
   +- LogicalRDD [city#124, country#125, population#126], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [city#112], [city#124], Inner
   :- Sort [city#112 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(city#112, 200), ENSURE_REQUIREMENTS, [id=#564]
   :     +- Filter isnotnull(city#112)
   :        +- Scan ExistingRDD[first_name#111,city#112]
   +- 

#### Notice how the parsed, analyzed, and optimized logical plans all contain ResolvedHint isBroadcastable=true because the broadcast() function was used. This hint isn’t included when the broadcast() function isn’t used.

## Automatic DetectionPermalink

- In many cases, Spark can automatically detect whether to use a broadcast join or not, depending on the size of the data. If Spark can detect that one of the joined DataFrames is small (10 MB by default), Spark will automatically broadcast it for us. The code below:

In [38]:
bigTable = spark.range(1, 100000000)
smallTable = spark.range(1, 10000) # size estimated by Spark - auto-broadcast
joinedNumbers = smallTable.join(bigTable, "id")
joinedNumbers.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner,Buffer(id))
:- Range (1, 10000, step=1, splits=Some(4))
+- Range (1, 100000000, step=1, splits=Some(4))

== Analyzed Logical Plan ==
id: bigint
Project [id#276L]
+- Join Inner, (id#276L = id#274L)
   :- Range (1, 10000, step=1, splits=Some(4))
   +- Range (1, 100000000, step=1, splits=Some(4))

== Optimized Logical Plan ==
Project [id#276L]
+- Join Inner, (id#276L = id#274L)
   :- Range (1, 10000, step=1, splits=Some(4))
   +- Range (1, 100000000, step=1, splits=Some(4))

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [id#276L]
   +- BroadcastHashJoin [id#276L], [id#274L], Inner, BuildLeft, false
      :- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [id=#585]
      :  +- Range (1, 10000, step=1, splits=4)
      +- Range (1, 100000000, step=1, splits=4)



- However, in the previous case, Spark did not detect that the small table could be broadcast. 
- The reason is that Spark will not determine the size of a local collection because it might be big, and evaluating its size may be an O(N) operation, which can defeat the purpose before any computation is made.

- Spark will perform auto-detection when
    - it constructs a DataFrame from scratch, e.g. spark.range
    - it reads from files with schema and/or size information, e.g. Parquet

## Configuring Broadcast Join Detection

- The threshold for automatic broadcast join detection can be tuned or disabled.
- The configuration is spark.sql.autoBroadcastJoinThreshold, and the value is taken in bytes. If you want to configure it to another number, we can set it in the SparkSession:

In [41]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 104857600)

- or deactivate it altogether by setting the value to -1.

In [39]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1) 

## Assignment

- Import the broadcast() method from pyspark.sql.functions.
- Create a new DataFrame broadcast_df by joining flights_df with airports_df, using the broadcasting.
- Show the query plan and consider differences from the original.

In [ ]:
# Import the broadcast method from pyspark.sql.functions
from ____ import ____

# Join the flights_df and airports_df DataFrames using broadcasting
broadcast_df = flights_df.____(____(airports_df), \
    flights_df["Destination Airport"] == airports_df["IATA"] )

# Show the query plan and compare against the original
broadcast_df.____()

- Creating a defined schema helps with data quality and import performance. As mentioned during the lesson, we'll create a simple schema to read in the following columns:

    - Name
    - Age
    - City

- The Name and City columns are StringType() and the Age column is an IntegerType().

- Import * from the pyspark.sql.types library.
- Define a new schema using the StructType method.
- Define a StructField for name, age, and city. Each field should correspond to the correct datatype and not be nullable.

In [ ]:
# Import the pyspark.sql.types library
____

# Define a new schema using the StructType method
people_schema = ____([
  # Define a StructField for each field
  StructField('name', ____, False),
  ____('____', IntegerType(), ____)
  ____
])

- Load the Data Frame.
- Add the transformation for F.lower() to the Destination Airport column.
- Drop the Destination Airport column from the Data Frame aa_dfw_df. Note the time for these operations to complete.
- Show the Data Frame, noting the time difference for this action to complete.

In [ ]:
# Load the CSV file
aa_dfw_df = ____.____.____('csv').options(Header=True).load('AA_DFW_2018.csv.gz')

# Add the airport column using the F.lower() method
aa_dfw_df = aa_dfw_df.withColumn('airport', ____(aa_dfw_df['Destination Airport']))

# Drop the Destination Airport column
aa_dfw_df = ____(aa_dfw_df['Destination Airport'])

# Show the DataFrame
____

- View the row count of df1 and df2.
- Combine df1 and df2 in a new DataFrame named df3 with the union method.
- Save df3 to a parquet file named AA_DFW_ALL.parquet.
- Read the AA_DFW_ALL.parquet file and show the count.

In [ ]:
# View the row count of df1 and df2
print("df1 Count: %d" % df1.____())
print("df2 Count: %d" % ____.____())

# Combine the DataFrames into one
df3 = df1.union(df2)

# Save the df3 DataFrame in Parquet format
df3.____.____('AA_DFW_ALL.parquet', mode='overwrite')

# Read the Parquet file into a new DataFrame and run a count
print(spark.read.____('AA_DFW_ALL.parquet').count())

- Import the AA_DFW_ALL.parquet file into flights_df.
- Use the createOrReplaceTempView method to alias the flights table.
- Run a Spark SQL query against the flights table.

In [ ]:
# Read the Parquet file into flights_df
flights_df = spark.read.____(____)

# Register the temp table
flights_df.____('flights')

# Run a SQL query of the average flight duration
avg_duration = spark.____('SELECT avg(flight_duration) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

## Comparing broadcast vs normal joins
- You've created two types of joins, normal and broadcasted. Now your manager would like to know what the performance improvement is by using Spark optimizations. If the results are promising, you'll be given more opportunity to tweak the Spark setup as needed.
- Your DataFrames normal_df and broadcast_df are available for your use.

- Execute .count() on the normal DataFrame.
- Execute .count() on the broadcasted DataFrame.
- Print the count and duration of the DataFrames noting and differences.

In [ ]:
start_time = time.time()
# Count the number of rows in the normal DataFrame
normal_count = ____
normal_duration = time.time() - start_time

start_time = time.time()
# Count the number of rows in the broadcast DataFrame
broadcast_count = ____
broadcast_duration = time.time() - start_time

# Print the counts and the duration of the tests
print("Normal count:\t\t%d\tduration: %f" % (normal_count, normal_duration))
print("Broadcast count:\t%d\tduration: %f" % (broadcast_count, broadcast_duration))

- Import the file 2015-departures.csv.gz to a DataFrame. Note the header is already defined.
- Filter the DataFrame to contain only flights with a duration over 0 minutes. Use the index of the column, not the column name (remember to use .printSchema() to see the column names / order).
- Add an ID column.
- Write the file out as a JSON document named output.json.

In [ ]:
# Import the data to a DataFrame
departures_df = spark.____(____, header=____)

# Remove any duration of 0
departures_df = departures_df.____(____)

# Add an ID column
departures_df = departures_df.____('id', ____)

# Write the file out to JSON format
____.write.____(____, mode='overwrite')
